# Description du Notebook

Ce notebook permet d'entraîner un modèle de NER en français avec spaC en se basant les infos de voyage. 

Les étapes incluent : 
- l'importation, le nettoyage et la préparation des données annotées,
- l'entraînement du modèle. 

Ensuite, on fait un comparaison de performances entre le modèle personnalisé et le modèle spaCy préentraîné, avec des visualisations pour évaluer la diversité et le nombre d'entités détectées par chaque modèle.


In [6]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
import time
import subprocess
import pandas as pd
import ast
from random import shuffle
import matplotlib.pyplot as plt
from collections import Counter
from spacy import displacy
from IPython.display import HTML

print("Librairies importées avec  succès")

Librairies importées avec  succès


In [7]:
models = {
    "sm": "fr_core_news_sm",
    "md": "fr_core_news_md",
    "lg": "fr_core_news_lg"
}
print("Chargement des modèles spaCy...")

# liste pour stocker les modèles chargés
loaded_models = []

for model_name, model_id in tqdm(models.items()):
    nlp = spacy.load(model_id)
    loaded_models.append((model_name, nlp))

print("Les modèles spaCy ont été bien chargés")

Chargement des modèles spaCy...


100%|██████████| 3/3 [00:13<00:00,  4.65s/it]

Les modèles spaCy ont été bien chargés


In [8]:
# dictionnaire pour stocker les modèles chargés
nlp_models = {}
for model_key, model_name in models.items():
    print(f"Chargement du modèle spaCy {model_key}...")
    nlp_models[model_key] = spacy.load(model_name)
    print(f"Modèle spaCy {model_key} chargé avec succès")

Chargement du modèle spaCy sm...
Modèle spaCy sm chargé avec succès
Chargement du modèle spaCy md...
Modèle spaCy md chargé avec succès
Chargement du modèle spaCy lg...
Modèle spaCy lg chargé avec succès


In [11]:
# test de NER sur du texte simple
def test_ner_recognition(nlp, text):
    print(f"Test de reconnaissance d'entités avec le modèle spaCy {nlp.meta['name']}...\n")
    doc = nlp(text)
    spacy.displacy.render(doc, style="ent", jupyter=True)

texte_simple = '''Je souhaiterais me rendre de Paris à Marseille.
Quelle est la meilleure façon d'aller à Sannois depuis la gare de Toulouse ?
Je prévois de rendre visite à mon ami Patrick à Paris en partant de Bordeaux.'''

for model_key in models.keys():
    test_ner_recognition(nlp_models[model_key], texte_simple)

Test de reconnaissance d'entités avec le modèle spaCy core_news_sm...



Test de reconnaissance d'entités avec le modèle spaCy core_news_md...



Test de reconnaissance d'entités avec le modèle spaCy core_news_lg...



In [13]:
# chargement du dataset
def load_dataset(file_path):
    print("Chargement du dataset pour l'entraînement...")
    
    # Read the CSV file in chunks to improve performance and memory usage
    chunksize = 10000  # Adjust chunk size as needed
    df_chunks = pd.read_csv(file_path, sep=',', encoding='utf-8', on_bad_lines='skip', chunksize=chunksize)
    
    # Concatenate chunks with a progress bar
    df = pd.concat(tqdm(df_chunks, total=len(pd.read_csv(file_path, nrows=10))))
    
    print("Dataset chargé avec succès.")
    return df

df = load_dataset('../../dataset/raw/initial_training_data.csv')

Chargement du dataset pour l'entraînement...


 10%|█         | 1/10 [00:00<00:00, 214.97it/s]

Dataset chargé avec succès.


In [14]:
# nettoyage des données
def clean_text(text):
    return text.replace('\r', '').strip()

df['text'] = df['text'].apply(clean_text)
print("Les données ont été nettoyées.")

Les données ont été nettoyées.


In [18]:
# vérification de l'alignement des entités nommées
def verify_entity_alignment(text, entities):
    verified_entities = []
    for ent in entities:
        start, end, label = ent['start'], ent['end'], ent['label']
        if text[start:end] == text[start:end]:
            verified_entities.append(ent)
        else:
            print(f"Skipping entity: {text[start:end]} in text: {text}")
    return verified_entities

In [19]:
# préparation des données pour spaCy
def prepare_data_for_spacy(df):
    print("Conversion du dataset au format spaCy...")
    db = DocBin()
    data = list(zip(df['text'], df['entities']))
    shuffle(data)

    for text, entities in data:
        entities = ast.literal_eval(entities)
        verified_entities = verify_entity_alignment(text, entities)

        if verified_entities:
            doc = nlp_models["sm"].make_doc(text)
            doc.ents = create_entity_spans(doc, verified_entities)
            db.add(doc)

    print("Dataset converti au format spaCy.")
    return db

In [20]:
# création des spans d'entités
def create_entity_spans(doc, verified_entities):
    ents = []
    for ent in verified_entities:
        start, end, label = ent['start'], ent['end'], ent['label']
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span:
            ents.append(span)
    return ents

print("Préparation des données pour spaCy...")
db = prepare_data_for_spacy(df)

Préparation des données pour spaCy...
Conversion du dataset au format spaCy...
Dataset converti au format spaCy.


In [21]:
# sauvegarde des données au format spaCy
db.to_disk("../../dataset/processed/processed_training_data.spacy")
print("Les données ont été sauvegardées")

Les données ont été sauvegardées


In [23]:
# entraînement du modèle
def train_model(config_path, output_path, training_data_path):
    print("Génération de la configuration...")
    subprocess.run(f"python -m spacy init config {config_path} --lang fr --pipeline ner --optimize efficiency --force", shell=True)
    print("Configuration générée.")

    print("Début de l'entraînement du modèle...")
    process = subprocess.Popen(f"python -m spacy train {config_path} --output {output_path} --paths.train {training_data_path} --paths.dev {training_data_path}", shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    with tqdm(total=100, desc="Entraînement du modèle") as pbar:
        while True:
            output = process.stdout.readline()
            if output == b"" and process.poll() is not None:
                break
            if output:
                print(output.decode().strip()) 
            pbar.update(1)
            time.sleep(1)

# entrainement de chaque modèle
for model_key in models.keys():
    print(f"Entraînement pour le modèle {model_key}...")
    train_model('./configuration.cfg', f'../../models/spacy/{model_key}', '../../dataset/processed/processed_training_data.spacy')
    print(f"Entraînement du modèle {model_key} terminé.")

Entraînement pour le modèle sm...
Génération de la configuration...
Configuration générée.
Début de l'entraînement du modèle...


Entraînement du modèle:   1%|          | 1/100 [00:03<05:51,  3.55s/it]

✔ Created output directory: ..\..\models\spacy\sm


Entraînement du modèle:   2%|▏         | 2/100 [00:04<03:21,  2.05s/it]

ℹ Saving to output directory: ..\..\models\spacy\sm


...

✔ Saved pipeline to output directory


Entraînement du modèle:  26%|██▌       | 26/100 [04:15<18:38, 15.11s/it]

..\..\models\spacy\lg\model-last


Entraînement du modèle:  26%|██▌       | 26/100 [04:16<12:09,  9.85s/it]

Entraînement du modèle lg terminé.


In [27]:
def load_and_test_model(model_path, text):
    nlp_model = spacy.load(model_path)
    doc = nlp_model(text)
    return doc

def test_model_on_texts(model_paths):
    texts = [
        '''Je souhaite me rendre à Lille en partant d'Aubervilliers pour assister à une conférence.''',
        '''Je dois planifier un voyage Nice Toulouse pour les prochaines vacances.''',
        '''Je veux partir à Paris à partir de Lyon via Marseille'''
    ]

    colors = {"DEPARTURE": "#1f77b4", "DESTINATION": "#2ca02c", "STOP": "#9467bd"}
    options = {"colors": colors}

    for model_key, model_path in model_paths.items():
        print(f"Test du modèle {model_key}...")
        for text in texts:
            doc = load_and_test_model(model_path, text)
            displacy.render(doc, style="ent", jupyter=True, options=options)

model_paths = {key: f"../../models/spacy/{key}/model-best/" for key in models.keys()}
test_model_on_texts(model_paths)

# Comparaison des modèles
def extract_entities(doc):
    return [(ent.text, ent.label_) for ent in doc.ents]

def compare_models(model_paths, texts, ground_truths):
    results = {key: [] for key in models.keys()}

    for text in texts:
        for model_key, model_path in model_paths.items():
            doc = load_and_test_model(model_path, text)
            entities = extract_entities(doc)
            results[model_key].append(entities)

    return results

texts_to_compare = [
    '''Je souhaite me rendre à Lille en partant d'Aubervilliers pour assister à une conférence.''',
    '''Je dois regarder les trains Toulouse - Brest pour aller voir mon ami Albert.'''
]

ground_truths = [
    [('Aubervilliers', 'DEPARTURE'), ('Lille', 'DESTINATION')],
    [('Toulouse', 'DEPARTURE'), ('Brest', 'DESTINATION')]
]

compare_models(model_paths, texts_to_compare, ground_truths)


Test du modèle sm...


C:\Users\vikne\anaconda3\Lib\site-packages\spacy\displacy\__init__.py:213: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  warnings.warn(Warnings.W006)


Test du modèle md...


Test du modèle lg...


{'sm': [[], [('Brest', 'DESTINATION')]],
 'md': [[], [('Brest', 'DESTINATION')]],
 'lg': [[], [('Brest', 'DESTINATION')]]}